In [1]:
from manim import *
import numpy as np

class DetailedDimensionScene(ThreeDScene):
    # Helper for 3D oriented labels
    def get_label3d(self, text, position, angle_x=0, angle_y=0, angle_z=0, scale=0.8, color=YELLOW):
        label = MathTex(text, color=color).scale(scale)
        label.move_to(ORIGIN)
        
        # Rotations to align text with the 3D planes
        if angle_x != 0: label.rotate(angle_x, axis=RIGHT)
        if angle_y != 0: label.rotate(angle_y, axis=UP)
        if angle_z != 0: label.rotate(angle_z, axis=OUT)
        
        label.move_to(position)
        return label

    def construct(self):
        # --- 1. CONFIGURATION ---
        val_r = 1.5;  str_r = "1.5"
        val_hc = 2.0; str_hc = "2"   # Height of Cylinder
        val_l = 4.0;  str_l = "4"
        val_b = 3.5;  str_b = "3.5"
        
        # Visual height vs Logical height
        visual_h_rect = 1.2; str_h_rect = "5" 

        opacity = 0.8
        
        # --- 2. CREATE SHAPES ---
        # Cuboid
        cuboid = Prism(dimensions=[val_l, val_b, visual_h_rect])
        cuboid.set_fill(BLUE, opacity=opacity).set_stroke(BLUE_E, width=2)
        
        # Cylinder
        cylinder = Cylinder(radius=val_r, height=val_hc, resolution=(32, 32))
        cylinder.set_fill(GREEN, opacity=opacity).set_stroke(GREEN_E, width=1)
        cyl_pos = np.array([0, 0, visual_h_rect/2 + val_hc/2])
        cylinder.move_to(cyl_pos)
        
        # Hemisphere
        hemisphere = Surface(
            lambda u, v: np.array([
                val_r * np.cos(u) * np.sin(v),
                val_r * np.sin(u) * np.sin(v),
                val_r * np.cos(v)
            ]),
            u_range=[0, 2 * PI], v_range=[0, PI / 2], resolution=(32, 32)
        )
        hemisphere.set_fill(RED, opacity=opacity).set_style(stroke_width=0.5, stroke_color=RED_E)
        hemi_pos = np.array([0, 0, visual_h_rect/2 + val_hc])
        hemisphere.shift(hemi_pos)

        # --- 3. SETUP SCENE ---
        axes = ThreeDAxes(x_length=6, y_length=6, z_length=5)
        self.set_camera_orientation(phi=70 * DEGREES, theta=45 * DEGREES)
        
        self.play(Create(axes), run_time=1)
        self.play(Create(cuboid), run_time=0.8)
        self.play(Create(cylinder), run_time=0.8)
        self.play(Write(hemisphere), run_time=0.8)
        self.wait(1)

        # --- 4. EXPLODE ---
        target_cyl = cyl_pos + np.array([0, 0, 2])
        target_hemi = hemi_pos + np.array([0, 0, 4])
        
        self.move_camera(
            frame_center=np.array([0,0,2]),
            added_anims=[
                cylinder.animate.move_to(target_cyl),
                hemisphere.animate.move_to(target_hemi)
            ],
            run_time=2
        )
        self.wait(0.5)

        # ==========================================
        #      PART A: CUBOID DIMENSIONS
        # ==========================================

        # 1. LENGTH
        self.move_camera(phi=75*DEGREES, theta=-90*DEGREES, zoom=1.5, frame_center=cuboid.get_center())
        
        p1 = np.array([-val_l/2, -val_b/2 - 0.2, -visual_h_rect/2])
        p2 = np.array([val_l/2, -val_b/2 - 0.2, -visual_h_rect/2])
        brace_l = DoubleArrow(p1, p2, buff=0, color=YELLOW)
        lbl_l = self.get_label3d(f"L = {str_l}", (p1+p2)/2 + np.array([0, -0.5, 0]), angle_x=90*DEGREES)
        
        self.play(GrowFromCenter(brace_l), Write(lbl_l))
        self.wait(1)

        # 2. BREADTH
        self.move_camera(phi=75*DEGREES, theta=0*DEGREES, zoom=1.5, frame_center=cuboid.get_center())

        p3 = np.array([val_l/2 + 0.2, -val_b/2, -visual_h_rect/2])
        p4 = np.array([val_l/2 + 0.2, val_b/2, -visual_h_rect/2])
        brace_b = DoubleArrow(p3, p4, buff=0, color=ORANGE)
        lbl_b = self.get_label3d(f"B = {str_b}", (p3+p4)/2 + np.array([0.5, 0, 0]), angle_x=90*DEGREES, angle_z=90*DEGREES, color=ORANGE)

        self.play(GrowFromCenter(brace_b), Write(lbl_b))
        self.wait(1)

        # 3. CUBOID HEIGHT
        self.move_camera(phi=80*DEGREES, theta=25*DEGREES, zoom=1.8)

        p_rect_bot = np.array([val_l/2, -val_b/2, -visual_h_rect/2])
        p_rect_top = np.array([val_l/2, -val_b/2, visual_h_rect/2])
        
        # Extension lines logic
        buff_h = 0.7
        p_rect_bot_ext = p_rect_bot + np.array([buff_h, -buff_h, 0])
        p_rect_top_ext = p_rect_top + np.array([buff_h, -buff_h, 0])

        ext_line_bot = Line(p_rect_bot, p_rect_bot_ext, color=WHITE, stroke_width=1)
        ext_line_top = Line(p_rect_top, p_rect_top_ext, color=WHITE, stroke_width=1)
        brace_h = DoubleArrow(p_rect_bot_ext, p_rect_top_ext, buff=0, color=WHITE, tip_length=0.2)
        lbl_h = self.get_label3d(f"H = {str_h_rect}", (p_rect_bot_ext + p_rect_top_ext)/2 + np.array([0.4, 0, 0]), angle_x=90*DEGREES, angle_z=90*DEGREES)

        self.play(Create(ext_line_bot), Create(ext_line_top), GrowFromCenter(brace_h), Write(lbl_h))
        self.wait(1.5)

        # ==========================================
        #      PART B: CYLINDER HEIGHT (NEW)
        # ==========================================
        
        # Move camera up to the cylinder
        self.move_camera(phi=80*DEGREES, theta=25*DEGREES, zoom=2.0, frame_center=target_cyl)

        # Calculate cylinder top/bottom points
        # Cylinder is at 'target_cyl'
        p_cyl_bot = target_cyl - np.array([0, 0, val_hc/2])
        p_cyl_top = target_cyl + np.array([0, 0, val_hc/2])
        
        # We need to extend lines OUT from the cylinder radius
        # The cylinder edge is at radius val_r. We want to start the extension line there.
        # We start drawing from (r, 0, z) and extend to (r + buff, 0, z)
        
        # Direction of extension: roughly along X axis because theta=25 keeps X-axis visible
        dir_vect = np.array([1, 0, 0]) 
        
        # Start points on the surface of the cylinder
        surf_bot = p_cyl_bot + dir_vect * val_r
        surf_top = p_cyl_top + dir_vect * val_r
        
        # End points for the dimension line (offset by buff_c)
        buff_c = 0.8
        ext_bot = surf_bot + dir_vect * buff_c
        ext_top = surf_top + dir_vect * buff_c
        
        # Create visual elements
        l_ext_c_bot = Line(surf_bot, ext_bot, color=WHITE, stroke_width=1)
        l_ext_c_top = Line(surf_top, ext_top, color=WHITE, stroke_width=1)
        
        brace_hc = DoubleArrow(ext_bot, ext_top, buff=0, color=GREEN_A, tip_length=0.2)
        
        # Label position: slightly right of the brace
        lbl_hc = self.get_label3d(f"h = {str_hc}", (ext_bot + ext_top)/2 + np.array([0.5, 0, 0]), 
                                  angle_x=90*DEGREES, angle_z=90*DEGREES, color=GREEN_A)

        self.play(
            Create(l_ext_c_bot), 
            Create(l_ext_c_top), 
            GrowFromCenter(brace_hc), 
            Write(lbl_hc)
        )
        self.wait(1.5)

        # ==========================================
        #      PART C: CYLINDER RADIUS (TOP VIEW)
        # ==========================================
        
        cyl_top_surface = target_cyl + np.array([0,0,val_hc/2])
        
        self.move_camera(
            phi=0*DEGREES, 
            theta=-90*DEGREES, 
            zoom=2.5, 
            frame_center=cyl_top_surface, 
            run_time=2.5
        )

        center_point = cyl_top_surface
        edge_point = cyl_top_surface + np.array([val_r, 0, 0])
        radius_line = Line(center_point, edge_point, color=YELLOW, stroke_width=4)
        
        lbl_r = self.get_label3d(f"r = {str_r}", (center_point+edge_point)/2 + np.array([0, 0.4, 0]), scale=1.2, color=YELLOW)
        
        self.play(Create(radius_line), Write(lbl_r))
        self.wait(3)

%manim -ql -v warning DetailedDimensionScene

Manim Community v0.19.1